In [ ]:
tweet_dir= Path("../data/processed/park_user_tweets")
results_city_type = {}

for city in tweet_dir.glob("*.json"):
    if city.stem not in results_city_type.keys():
        with open(city, 'r') as f:
            park_user_tweets = json.load(f)
        print("City: {} w {}".format(city, len(park_user_tweets)))
        tweets_w_parks = get_time_control_text_park(park_user_tweets)
        tweet_df = pd.DataFrame.from_dict(tweets_w_parks,dtype=str)
        tweet_df = pd.merge(tweet_df,park_cols,how='left',on='park_id')
        cats = list(set(tweet_df.size_cat.values))
        results_type = {}
        park_list = list(set(tweet_df.park_name))
        park_stops  = get_park_stopwords(park_list)
        stop_list = park_stops+stop_dict['all_cities']+ stop_dict[city.stem]
        for cat in cats:
            tweet_subset = tweet_df[tweet_df.size_cat ==cat]
            park_text = list(tweet_subset.park_text.values)
            control_text = list(tweet_subset.control_text.values)
            results_type[cat] = bootstrap_sentiment(park_text, control_text, stop_list,sample=.8,runs=5)
        results_city_type[city.stem] = results_type